In [50]:
# Comments
# Workflow is stop-start at best and urgently needs to be revisited, to be standardized, and to be automated
# In current form:
# # Run function on existing Excel spreadsheet
# # spreadsheet MUST be tab-delimited; this was added ad-hoc to fix problems with commas
# # note that there are other better solutions, such as using read.csv()
# # a column of hash '#' marks MUST be added at the end of samples; this was added ad-hoc due to a parsing conflict
# # other solutions exist, such as parsing by line or parsing by column count
# # Then, the data will be a pandas dataframe, which will be output to a csv file.
# # This csv file must be manually parsed with two steps:
# # # remove duplicate marker "_i" where "i" is an incrementer; this obviously can be modified multiple ways
# # # Scan the file to rename duplicated Haplotype entries (but DO NOT MERGE THE ENTRIES--having replicated entries is critical!), for example "A001-A008" is the same as "A008-A001"; again, this obviously can be modified multiple ways
# # This output is ready for training, provided it is in the format of Haplotypes as the rownames and alleles as the column names across the top

In [1]:
# Functions

import re
import time
import pandas as pd

def openFileAsList(f):
    l = []
    with open(f, 'r') as fOpen:
        for i in fOpen:
            i = i.rstrip('\r\n')
            l.append(i)
    return l

def openAndParse(f):
    hapA = []
    hapB = []
    hapList = []
    with open(f, 'r') as fOpen:
        importData = False
        for i in fOpen:
            if importData:
                pLine = i.split('#')
                pLineD = pLine[0].split('\t')
                hapList.append(pLineD)
            else:
                # i = i.rstrip('\r\n')
                # m = re.search('run_id', i)
                m = re.search('Comments', i)
                m2 = re.search('MHC', i)
                if m:
                    importData = True
                    continue
                if m2:
                    iParsed = re.sub('".*"', '-', i) # removes error entries from consideration
                    # the above parsing will fail on some files, unknown why, probably encoding.  Fix in TextEdit.
                    p = iParsed.split('-')
                    pLine = iParsed.split('#')
                    pLineD = pLine[0].split('\t')
                    if p[1][0] == 'A':
                        hapA.append(pLineD[1:])
                    if p[1][0] == 'B':
                        hapB.append(pLineD[1:])
    return ((hapA, hapB), hapList)

def mapAndIndex(l):
    retList = []
    lMap = list(map(lambda x: x!= '', l))
    for a,b in enumerate(lMap):
        if b:
            retList.append(a)
    return retList

def parseAndMerge(parseList, mergeList):
    mergeReturnList = []
    for tItem in mergeList:
        mergeReturnList.append((tItem, []))
    for x in range(0, len(parseList)):
        hList = parseList[x]
        hValue = hList[0]
        hList = hList[1:]
        # python copies by reference:
        # do not modify list mid-loop or unpredictable things may occur
        hListWithIndexes = mapAndIndex(hList)
        for idx in hListWithIndexes:
            mergeReturnList[idx][1].append(hValue)
    return mergeReturnList
def parseAlleleToList(alleleList):
    rListAsList = []
    for itm in alleleList:
        s = itm.split('_')
        s_tmp = '_'.join(s[1:])
        rListAsList.append(s_tmp)
    return rListAsList
def parseAlleleString(alleleList):
    # deprecated
    rListAsString = ''
    for itm in alleleList:
        s = itm.split('_')
        s_tmp = '_'.join(s[1:])
        rListAsString += s_tmp + '\n'
    rListAsString = rListAsString[:-1]
    return rListAsString
def parseAllelesToString(alleleList):
    rListAsString = ''
    print(alleleList)
    for itm in alleleList:
        s = str(itm[0]) + ',' + str(itm[1])
        rListAsString += s + '\n'
    return rListAsString

def parseAlleleStringPandas(alleleList, allAlleles):
    alleleTupleList = []
    for a in allAlleles:
        if a not in alleleList:
            # alleleTupleList.append((a, 0))
            alleleTupleList.append(0)
        else:
            # alleleTupleList.append((a, 1))
            alleleTupleList.append(1)
    return alleleTupleList

In [3]:
allAlleleList = openFileAsList('alleles_parsed.txt')
hapAandB2 = openAndParse('20929_MiSeq137_AR-Indian36_GS-Genotyper_RW_upd.txt')
print(hapAandB2)
hapA = hapAandB2[0][0]
hapB = hapAandB2[0][1]
# print(hapA)
hapResults = hapAandB2[1]
# print(hapResults)

        
hapResultsA_Tuple = []
hapResultsB_Tuple = []

for x in range(0, len(hapA[0])):
    hapResultsA_Tuple.append((hapA[0][x], hapA[1][x]))
for x in range(0, len(hapB[0])):
    hapResultsB_Tuple.append((hapB[0][x], hapB[1][x]))
hResultsList = []
hapResultsA_TupleList = parseAndMerge(hapResults, hapResultsA_Tuple)
hapResultsB_TupleList = parseAndMerge(hapResults, hapResultsB_Tuple)
alleleDF = None
columns = []
for nItem in hapResultsA_TupleList:
    # print('\n')
    if nItem[0][0] == nItem[0][1]:
        columns.append(nItem[0][0])
    else:
        cString = str(nItem[0][0] + '-' + nItem[0][1])
        columns.append(cString)
columns = columns[:-1]
iterValue = 1
for tItem in hapResultsA_TupleList:
    cString = str(tItem[0][0] + '-' + tItem[0][1])
    lOutput = parseAlleleToList(tItem[1])
    tupleOutput = parseAlleleStringPandas(lOutput, allAlleleList)
    if alleleDF is None:
        pdDict = dict()
        pdDict[cString] = tupleOutput
        alleleDF = pd.DataFrame.from_dict(pdDict, orient='index').transpose()
        alleleDF.index=allAlleleList
        alleleDF.index.name = 'allele'
    else:
        # print('\n')
        if tItem[0][0] == tItem[0][1]:
            # print(tItem[0][0])
            cString = tItem[0][0]
            pdDict = dict()
            # if cString in alleleDF:
            #    cString = cString + '_' + str(iterValue)
            #    iterValue += 1
            pdDict[cString] = tupleOutput
            alleleDFnew = pd.DataFrame.from_dict(pdDict, orient='index').transpose()
            alleleDFnew.index=allAlleleList
            alleleDFnew.index.name = 'allele'
            alleleDFres = pd.concat([alleleDF, alleleDFnew], axis=1, join_axes=[alleleDF.index])
            alleleDF = alleleDFres
            # sOutput = parseAllelesToString(tupleOutput)
            # print(sOutput)
        else:
            # cString = str(tItem[0][0] + '-' + tItem[0][1])
            cStringRev = str(tItem[0][1] + '-' + tItem[0][0])
            if cStringRev in alleleDF:
                cString = cStringRev
            pdDict = dict()
            cStringRev = str(tItem[0][1] + '-' + tItem[0][0])
            pdDict[cString] = tupleOutput
            alleleDFnew = pd.DataFrame.from_dict(pdDict, orient='index').transpose()
            alleleDFnew.index=allAlleleList
            alleleDFnew.index.name = 'allele'
            alleleDFres = pd.concat([alleleDF, alleleDFnew], axis=1, join_axes=[alleleDF.index])
            alleleDF = alleleDFres
alleleDF.to_csv('allele_df-20929.hapA.csv')

(([['A004', 'A004', 'A004', 'A004', 'A004', 'A004', 'A004', 'A008', 'A004', 'A004', 'A002a', 'A002a', 'A001', 'A002a', 'A001', 'A002a', 'A002a', 'A001', 'A002a', 'A002a', 'A001', 'A004', 'A002a', 'A002a', 'A008', 'A006', 'A004', 'A004', 'A002a', 'A004', 'A004', 'A004', 'A004', 'A002a', 'A002a', 'A004', 'A001', 'A001', 'A001', 'A001', 'A001', 'A004', 'A001', 'A004', 'A004', 'A004', 'A004', '-', 'A012', '-', 'A001', 'A004', 'A004', 'A023', 'A002a', 'A004', 'A023', 'A008', 'A006', 'A004', 'A004', 'A002a', 'A004', 'A004', '-', 'A004', 'A004', 'A001', 'A001', 'A002a', 'A002a', 'A002a', 'A002a', 'A002a', 'A001', 'A001', 'A004', 'A002a', 'A002a', 'A008', 'A002a', 'A004', 'A004', '-', 'A004', 'A012', '-', '-', 'A001', '-', '-', 'A023', '-', 'A002a', 'A004', 'A004', ''], ['A025', 'A001', 'A001', 'A004', 'A008', 'A002a', 'A004', 'A004', 'A023', 'A074', 'A008', 'A008', 'A023', 'A004', 'A004', 'A004', '-', 'A004', 'A008', 'A023', 'A002a', 'A008', 'A004', '-', 'A025', 'A008', 'A008', 'A025', 'A004'

In [21]:
s = 'Hello World'
s[0:3]

'Hel'